In [1]:
import numpy as np
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.preprocessing import LabelEncoder
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from itertools import combinations


In [2]:

# Example DataFrame with columns X, Y, Z, Atom_Name, Residue_Name, Residue_ID, Atom_Type, and Timeframe
# Load nodes for all timesteps
filepath = "/home/mhanowar/Downloads/HB1000frames.csv"  # Replace with the actual file path
data = pd.read_csv(filepath)


In [3]:
# Filter the dataframe based on Timeframe
df = data.loc[data['Timeframe'] < 255006].copy()
df.rename(columns={"Timeframe": "time"}, inplace=True)

# Add a combined 'node' column
df['node'] = df.apply(lambda row: f"{row['Atom_Name']}_{row['Residue_ID']}", axis=1)
# Display the updated dataframe
df


,X,Y,Z,Atom_Name,Residue_Name,Residue_ID,Atom_Type,time,node
0,12.759892,2.253709,33.260902,C1,CSP,1,cb,255000,C1_1
1,12.862613,3.581455,34.023949,C2,CSP,1,cb,255000,C2_1
2,11.457548,4.321817,34.003315,C3,CSP,1,cb,255000,C3_1
3,10.981806,4.421790,32.537914,C4,CSP,1,cb,255000,C4_1
4,11.038748,3.091581,31.915064,O5,CSP,1,ob,255000,O5_1
...,...,...,...,...,...,...,...,...,...
36751,21.767370,65.705544,49.322563,H8,SFL,14,ha,255005,H8_14
36752,22.807545,63.486736,48.436836,H9,SFL,14,ha,255005,H9_14
36753,19.063622,56.540447,53.720280,H10,SFL,14,ha,255005,H10_14
36754,22.261448,58.099701,56.088982,H11,SFL,14,ha,255005,H11_14


In [4]:
#Select rows where Residue_ID is 5 and Atom_Type is either 'o' or 'os'
# df1 = df[(df['Residue_ID'] == 5) & df['Atom_Type'].isin(['o', 'os'])].reset_index(drop=True)
# Select all O Atoms
# Select relevant atom types and residue names for calculations
df1 = df[df['Atom_Name'].str.startswith('O')].reset_index(drop=True)
df2 = df[(df['Residue_Name'] == 'CSP') & (df['Atom_Type'] == 'n')].reset_index(drop=True)
df3 = df[(df['Residue_Name'] == 'CSP') & (df['Atom_Type'] == 'hn')].reset_index(drop=True)

In [5]:
# Helper functions
def calculate_angle(vec1, vec2):
    """Calculate angle between two vectors."""
    cos_theta = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    angle_rad = np.arccos(np.clip(cos_theta, -1.0, 1.0))
    return np.degrees(angle_rad)

def calculate_distance(row1, row2):
    """Calculate Euclidean distance."""
    return np.sqrt((row1['X'] - row2['X'])**2 + (row1['Y'] - row2['Y'])**2 + (row1['Z'] - row2['Z'])**2)

# Calculate angles and distances
results = []

for t in df1['time'].unique():
    df1_time = df1[df1['time'] == t].reset_index(drop=True)
    df2_time = df2[df2['time'] == t].reset_index(drop=True)
    df3_time = df3[df3['time'] == t].reset_index(drop=True)
    
    for _, row1 in df1_time.iterrows():
        for idx, row3 in df3_time.iterrows():
            row2 = df2_time.loc[idx]
            
            # Vectors and calculations
            vec3_to_df1 = np.array([row1['X'] - row3['X'], row1['Y'] - row3['Y'], row1['Z'] - row3['Z']])
            vec3_to_df2 = np.array([row2['X'] - row3['X'], row2['Y'] - row3['Y'], row2['Z'] - row3['Z']])
            angle = calculate_angle(vec3_to_df1, vec3_to_df2)
            distance = calculate_distance(row1, row2)
            
            # Append results
            results.append({'src': row1['node'], 'dst': row2['node'], 'time': t, 'angle': 'NaN', 'distance': distance})
            results.append({'src': row1['node'], 'dst': row3['node'], 'time': t, 'angle': angle, 'distance': 'NaN'})

# Convert results to DataFrame
dist_angle_df = pd.DataFrame(results)
dist_angle_df

,src,dst,time,angle,distance
0,O5_1,N11_1,255000,NaN,4.435484
1,O5_1,H9_1,255000,61.040505,NaN
2,O5_1,N23_1,255000,NaN,5.677049
3,O5_1,H19_1,255000,73.52471,NaN
4,O5_1,N35_1,255000,NaN,6.328246
...,...,...,...,...,...
1544827,O2_14,H638_4,255005,86.2119,NaN
1544828,O2_14,N771_4,255005,NaN,84.204845
1544829,O2_14,H648_4,255005,129.928505,NaN
1544830,O2_14,N783_4,255005,NaN,83.572247


In [6]:
# Add molecular IDs
dist_angle_df['src_mol'] = dist_angle_df['src'].apply(lambda x: int(x.split('_')[1]))
dist_angle_df['dst_mol'] = dist_angle_df['dst'].apply(lambda x: int(x.split('_')[1]))

# Create 'nh_id' column with alternating values 1, 1, 2, 2, ...
dist_angle_df['nh_id'] = (np.arange(len(dist_angle_df)) // 2) + 1

# Display the updated DataFrame
dist_angle_df


,src,dst,time,angle,distance,src_mol,dst_mol,nh_id
0,O5_1,N11_1,255000,NaN,4.435484,1,1,1
1,O5_1,H9_1,255000,61.040505,NaN,1,1,1
2,O5_1,N23_1,255000,NaN,5.677049,1,1,2
3,O5_1,H19_1,255000,73.52471,NaN,1,1,2
4,O5_1,N35_1,255000,NaN,6.328246,1,1,3
...,...,...,...,...,...,...,...,...
1544827,O2_14,H638_4,255005,86.2119,NaN,14,4,772414
1544828,O2_14,N771_4,255005,NaN,84.204845,14,4,772415
1544829,O2_14,H648_4,255005,129.928505,NaN,14,4,772415
1544830,O2_14,N783_4,255005,NaN,83.572247,14,4,772416


In [ ]:
# Convert 'distance' and 'angle' columns to numeric, coercing errors to NaN
dist_angle_df['distance'] = pd.to_numeric(dist_angle_df['distance'], errors='coerce')
dist_angle_df['angle'] = pd.to_numeric(dist_angle_df['angle'], errors='coerce')

# Filter based on conditions
filtered_df = dist_angle_df[((dist_angle_df['distance'] <= 5) & (~dist_angle_df['distance'].isna())) | 
                            ((dist_angle_df['angle'] >= 135) & (dist_angle_df['angle'] < 180) & (~dist_angle_df['angle'].isna()))]

# Add 'label' column based on HB conditions
filtered_df['label'] = filtered_df.apply(
    lambda row: 2 if 5 <= row['src_mol'] <= 14 and 1 <= row['dst_mol'] <= 4 else 1, axis=1
)

# Reset index
filtered_df.reset_index(drop=True, inplace=True)

# Display the filtered DataFrame
filtered_df

/tmp/ipykernel_3785534/3772779069.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['class'] = filtered_df.apply(


,src,dst,time,angle,distance,src_mol,dst_mol,nh_id,class
0,O5_1,N11_1,255000,NaN,4.435484,1,1,1,1
1,O5_1,N79_1,255000,NaN,3.043924,1,1,6,1
2,O5_1,H83_1,255000,172.370689,NaN,1,1,7,1
3,O5_1,H177_1,255000,151.039706,NaN,1,1,15,1
4,O5_1,H194_1,255000,167.839722,NaN,1,1,16,1
...,...,...,...,...,...,...,...,...,...
118842,O2_14,H288_4,255005,145.158506,NaN,14,4,772386,2
118843,O2_14,H342_4,255005,148.646259,NaN,14,4,772390,2
118844,O2_14,H352_4,255005,142.905322,NaN,14,4,772391,2
118845,O2_14,H453_4,255005,160.960087,NaN,14,4,772399,2


In [8]:

# Encode nodes into unique integers
le_node1, le_node2, le_node3 = LabelEncoder(), LabelEncoder(), LabelEncoder()
filtered_df['src'] = le_node1.fit_transform(filtered_df['src'])
filtered_df['dst'] = le_node2.fit_transform(filtered_df['dst']) + filtered_df['src'].max() + 1
filtered_df.head(5)


/tmp/ipykernel_3785534/2825895535.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['src'] = le_node1.fit_transform(filtered_df['src'])
/tmp/ipykernel_3785534/2825895535.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['dst'] = le_node2.fit_transform(filtered_df['dst']) + filtered_df['src'].max() + 1


,src,dst,time,angle,distance,src_mol,dst_mol,nh_id,class
0,416,807,255000,NaN,4.435484,1,1,1,1
1,416,1011,255000,NaN,3.043924,1,1,6,1
2,416,791,255000,172.370689,NaN,1,1,7,1
3,416,618,255000,151.039706,NaN,1,1,15,1
4,416,622,255000,167.839722,NaN,1,1,16,1


In [9]:
# Extract src and dst column features
src_feats = filtered_df[['src', 'src_mol']].rename(columns={'src': 'node', 'src_mol': 'mol'})
dst_feats = filtered_df[['dst', 'dst_mol']].rename(columns={'dst': 'node', 'dst_mol': 'mol'})

# Concatenate src and dst features
node_feats = pd.concat([src_feats, dst_feats])

# Drop duplicates
node_feats = node_feats.drop_duplicates().reset_index(drop=True)

# Save to CSV
node_feats.to_csv('DATA/HB/node_feats.csv', index=False)

# Display the node features
print(node_feats)

      node  mol
0      416    1
1      580    1
2        4    1
3       90    1
4      110    1
...    ...  ...
1014   671    4
1015   769    2
1016   656    4
1017   663    4
1018   784    1

[1019 rows x 2 columns]


In [ ]:
# Add ext_roll column
new_df2 = filtered_df.copy()
num_rows = len(new_df2 )

# Ensure 'ext_roll' is initialized with zeros
new_df2 ['ext_roll'] = 0

# Assign 1 to the middle 15% rows
new_df2.loc[int(num_rows * 0.7):int(num_rows * 0.85), 'ext_roll'] = 1

# Assign 2 to the last 15% rows
new_df2.loc[int(num_rows * 0.85):, 'ext_roll'] = 2

# Insert an 'idx' column at the beginning
new_df2.insert(0, 'idx', range(len(filtered_df)))

# Reindex and retain only the required columns
new_df2 = new_df2[['idx', 'src', 'dst', 'time', 'label', 'ext_roll', 'nh_id']]

# Display the first few rows of the dataframe
print(new_df2.head(50))


# # Save the updated dataframe to a CSV file
new_df2.to_csv('DATA/HB/dataset.csv', index=False)

    idx  src   dst    time  class  ext_roll  nh_id
0     0  416   807  255000      1         0      1
1     1  416  1011  255000      1         0      6
2     2  416   791  255000      1         0      7
3     3  416   618  255000      1         0     15
4     4  416   622  255000      1         0     16
5     5  416   649  255000      1         0     22
6     6  416   664  255000      1         0     25
7     7  416   679  255000      1         0     29
8     8  416   711  255000      1         0     37
9     9  416   723  255000      1         0     39
10   10  416   761  255000      1         0     48
11   11  416   772  255000      1         0     51
12   12  416   792  255000      1         0     61
13   13  416   796  255000      1         0     62
14   14  416   597  255000      1         0     63
15   15  416   608  255000      1         0     66
16   16  416   658  255000      1         0     78
17   17  416   712  255000      1         0     91
18   18  416   716  255000     

In [11]:
# # Get unique nodes from node_1, node_2, and node_3 columns
# unique_nodes_1 = filtered_df['src'].unique()
# unique_nodes_2 = filtered_df['dst'].unique()

# # Assign a color for each group (e.g., tab10 colormap)
# color_1 = plt.cm.tab10(0)  # Color for node_1 group
# color_2 = plt.cm.tab10(1)  # Color for node_2 group
# color_3 = plt.cm.tab10(2)  # Color for node_3 group

# # Create a combined color map based on the node groups
# node_colors = {}
# for node in unique_nodes_1:
#     node_colors[node] = color_1
# for node in unique_nodes_2:
#     node_colors[node] = color_2


# # Create a graph
# G = nx.Graph()

# # Add edges between node_1 and node_2
# edges_1_2 = filtered_df[['node_1', 'node_2']].values.tolist()
# G.add_edges_from(edges_1_2)

# # Add edges between node_1 and node_3
# edges_1_3 = filtered_df[['node_1', 'node_3']].values.tolist()
# G.add_edges_from(edges_1_3)

# # Add edges between node_2 and node_3 (these will be bold)
# edges_2_3 = filtered_df[['node_2', 'node_3']].values.tolist()
# G.add_edges_from(edges_2_3)

# # Define the layout for the graph
# pos = nx.spring_layout(G)

# # Separate edges into normal and bold
# normal_edges = edges_1_2 + edges_1_3
# bold_edges = edges_2_3

# # Plot the graph
# plt.figure(figsize=(8, 4))
# nx.draw(
#     G,
#     pos,
#     with_labels=True,
#     node_size=500,
#     node_color=[node_colors[node] for node in G.nodes()],
#     font_size=10,
#     font_weight='bold',
#     edge_color='gray',
#     edgelist=normal_edges,
#     width=1  # Normal edges width
# )

# # Draw the bold edges separately
# nx.draw_networkx_edges(
#     G,
#     pos,
#     edgelist=bold_edges,
#     width=2,  # Bold edges width
#     edge_color='black'
# )

# plt.title('Graph')
# plt.show()
